In [12]:
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# Parameters

modes = ['triplet','eng_mfcc_env','eng_all_classall','eng_all_classred','eng_all_syllall','eng_all_syllred','eng_all_phonall','eng_all_phonred',
         'vae','classall','classred','syllall','syllred','phonall','phonred'] # Triplet!!
#clfs = ['slp','mlp','logr','knn','rf','xgboost']
clfs = ['knn']

# Calculate utterance-wise weights

num_test_utterances = []
for part in range(28):
    if part<=9:
        test_dataset = np.load('../../data/interim/AVP/Dataset_Test_0' + str(part) + '_1024.npy')
    else:
        test_dataset = np.load('../../data/interim/AVP/Dataset_Test_' + str(part) + '_1024.npy')
    num_test_utterances.append(test_dataset.shape[0])
utterance_wise_weights = num_test_utterances/np.sum(np.array(num_test_utterances))

In [14]:
# Results

for c in range(len(clfs)):

    for b in range(len(modes)):

        mode = modes[b]

        print([mode,clfs[c]])

        accuracies = np.load('../../results/' + mode + '/accuracies.npy')

        frame_sizes = ['1024']

        accuracies = np.mean(accuracies,axis=-1)
        accuracies_allpart = np.mean(accuracies,axis=-2)
        
        for a in range(len(frame_sizes)):
            print(accuracies_allpart[a,c])

['triplet', 'knn']
0.6019977856997041
['eng_mfcc_env', 'knn']
0.6860734769927309
['eng_all_classall', 'knn']
0.729527762477732
['eng_all_classred', 'knn']
0.7257496022446432
['eng_all_syllall', 'knn']
0.6654909759597343
['eng_all_syllred', 'knn']
0.6877446753911499
['eng_all_phonall', 'knn']
0.625166814902711
['eng_all_phonred', 'knn']
0.5940795153227418
['vae', 'knn']
0.7086937351482387
['classall', 'knn']
0.6999708219327253
['classred', 'knn']
0.6289770305135116
['syllall', 'knn']
0.76549887867688
['syllred', 'knn']
0.7711745997627661
['phonall', 'knn']
0.7581311746567713
['phonred', 'knn']
0.7696244369456047


In [5]:
# Results Also Valid

for c in range(len(clfs)):

    for b in range(len(modes)):

        mode = modes[b]

        print([mode,clfs[c]])

        accuracies = np.load('../../results/' + mode + '/accuracies_also_valid.npy')

        frame_sizes = ['1024']

        accuracies = np.mean(accuracies,axis=-1)
        accuracies_allpart = np.mean(accuracies,axis=-2)
        
        for a in range(len(frame_sizes)):
            print(accuracies_allpart[a,c])

['eng_mfcc_env', 'knn']
0.6909366094279135
['eng_all_classall', 'knn']
0.7312142563417295
['eng_all_classred', 'knn']
0.7257325368151749
['eng_all_syllall', 'knn']
0.6588745201564177
['eng_all_syllred', 'knn']
0.7014448226052783
['eng_all_phonall', 'knn']
0.6229397616965174
['eng_all_phonred', 'knn']
0.5857932465749103
['vae', 'knn']
0.7109027477813896
['classall', 'knn']
0.7048480782052279
['classred', 'knn']
0.6344616406908206
['syllall', 'knn']
0.7754312892352767
['syllred', 'knn']
0.7816773713787514
['phonall', 'knn']
0.754027948072874
['phonred', 'knn']
0.7603867673637985


In [18]:
# Results Utterance-Wise

for c in range(len(clfs)):

    for b in range(len(modes)):

        mode = modes[b]

        print([mode,clfs[c]])

        accuracies = np.load('../../results/' + mode + '/accuracies.npy')

        frame_sizes = ['1024']

        accuracies = np.mean(accuracies,axis=-1)
        accuracies[0,:,0] *= utterance_wise_weights
        accuracies_allpart = np.sum(accuracies,axis=-2)
        
        for a in range(len(frame_sizes)):
            print(accuracies_allpart[a,c])

['eng_mfcc_env', 'knn']
0.6640435835351091
['eng_all_classall', 'knn']
0.6930992736077481
['eng_all_classred', 'knn']
0.6930992736077481
['eng_all_syllall', 'knn']
0.6628329297820824
['eng_all_syllred', 'knn']
0.6761501210653752
['eng_all_phonall', 'knn']
0.6016949152542372
['eng_all_phonred', 'knn']
0.5738498789346247
['vae', 'knn']
0.7039951573849879
['classall', 'knn']
0.6918886198547216
['classred', 'knn']
0.6204600484261502
['syllall', 'knn']
0.7687651331719128
['syllred', 'knn']
0.7512106537530266
['phonall', 'knn']
0.75544794188862
['phonred', 'knn']
0.74455205811138


In [19]:
# Results Utterance-Wise Also Valid

for c in range(len(clfs)):

    for b in range(len(modes)):

        mode = modes[b]

        print([mode,clfs[c]])

        accuracies = np.load('../../results/' + mode + '/accuracies_also_valid.npy')

        frame_sizes = ['1024']

        accuracies = np.mean(accuracies,axis=-1)
        accuracies[0,:,0] *= utterance_wise_weights
        accuracies_allpart = np.sum(accuracies,axis=-2)
        
        for a in range(len(frame_sizes)):
            print(accuracies_allpart[a,c])

['eng_mfcc_env', 'knn']
0.679176755447942
['eng_all_classall', 'knn']
0.7052058111380145
['eng_all_classred', 'knn']
0.7015738498789346
['eng_all_syllall', 'knn']
0.6700968523002422
['eng_all_syllred', 'knn']
0.6846246973365617
['eng_all_phonall', 'knn']
0.608958837772397
['eng_all_phonred', 'knn']
0.5865617433414043
['vae', 'knn']
0.7106537530266344
['classall', 'knn']
0.7003631961259081
['classred', 'knn']
0.6222760290556901
['syllall', 'knn']
0.7717917675544794
['syllred', 'knn']
0.7651331719128329
['phonall', 'knn']
0.7590799031476998
['phonred', 'knn']
0.7487893462469735


In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Parameters

modes = ['eng_mfcc_env','eng_all_classall','eng_all_classred','eng_all_syllall','eng_all_syllred','eng_all_phonall','eng_all_phonred',
         'vae','classall','classred','syllall','syllred','phonall','phonred','triplet'] # Triplet!!
#clfs = ['mlp','logr','knn','rf','xgboost']
clfs = ['knn']

# Calculate utterance-wise weights

num_test_utterances = []
for part in range(28):
    if part<=9:
        test_dataset = np.load('../../data/interim/AVP/Dataset_Test_0' + str(part) + '_1024.npy')
    else:
        test_dataset = np.load('../../data/interim/AVP/Dataset_Test_' + str(part) + '_1024.npy')
    num_test_utterances.append(test_dataset.shape[0])
utterance_wise_weights = num_test_utterances/np.sum(np.array(num_test_utterances))

In [4]:
# Results

for c in range(len(clfs)):

    for b in range(len(modes)):

        mode = modes[b]

        print([mode,clfs[c]])

        accuracies = np.load('../../results/' + mode + '/accuracies.npy')

        frame_sizes = ['1024']

        accuracies = np.mean(accuracies,axis=-1)
        accuracies_allpart = np.mean(accuracies,axis=-2)
        
        for a in range(len(frame_sizes)):
            print(accuracies_allpart[a,c])

['eng_mfcc_env', 'knn']
0.6860734769927309
['eng_all_classall', 'knn']
0.729527762477732
['eng_all_classred', 'knn']
0.7257496022446432
['eng_all_syllall', 'knn']
0.6654909759597343
['eng_all_syllred', 'knn']
0.6877446753911499
['eng_all_phonall', 'knn']
0.625166814902711
['eng_all_phonred', 'knn']
0.5940795153227418
['vae', 'knn']
0.7086937351482387
['classall', 'knn']
0.6999708219327253
['classred', 'knn']
0.6289770305135116
['syllall', 'knn']
0.76549887867688
['syllred', 'knn']
0.7711745997627661
['phonall', 'knn']
0.7581311746567713
['phonred', 'knn']
0.7696244369456047
['triplet', 'knn']
0.6192411985743647


In [5]:
# Results Also Valid

for c in range(len(clfs)):

    for b in range(len(modes)):

        mode = modes[b]

        print([mode,clfs[c]])

        accuracies = np.load('../../results/' + mode + '/accuracies_also_valid.npy')

        frame_sizes = ['1024']

        accuracies = np.mean(accuracies,axis=-1)
        accuracies_allpart = np.mean(accuracies,axis=-2)
        
        for a in range(len(frame_sizes)):
            print(accuracies_allpart[a,c])

['eng_mfcc_env', 'knn']
0.6909366094279135
['eng_all_classall', 'knn']
0.7312142563417295
['eng_all_classred', 'knn']
0.7257325368151749
['eng_all_syllall', 'knn']
0.6588745201564177
['eng_all_syllred', 'knn']
0.7014448226052783
['eng_all_phonall', 'knn']
0.6229397616965174
['eng_all_phonred', 'knn']
0.5857932465749103
['vae', 'knn']
0.7109027477813896
['classall', 'knn']
0.7048480782052279
['classred', 'knn']
0.6344616406908206
['syllall', 'knn']
0.7754312892352767
['syllred', 'knn']
0.7816773713787514
['phonall', 'knn']
0.754027948072874
['phonred', 'knn']
0.7603867673637985


In [18]:
# Results Utterance-Wise

for c in range(len(clfs)):

    for b in range(len(modes)):

        mode = modes[b]

        print([mode,clfs[c]])

        accuracies = np.load('../../results/' + mode + '/accuracies.npy')

        frame_sizes = ['1024']

        accuracies = np.mean(accuracies,axis=-1)
        accuracies[0,:,0] *= utterance_wise_weights
        accuracies_allpart = np.sum(accuracies,axis=-2)
        
        for a in range(len(frame_sizes)):
            print(accuracies_allpart[a,c])

['eng_mfcc_env', 'knn']
0.6640435835351091
['eng_all_classall', 'knn']
0.6930992736077481
['eng_all_classred', 'knn']
0.6930992736077481
['eng_all_syllall', 'knn']
0.6628329297820824
['eng_all_syllred', 'knn']
0.6761501210653752
['eng_all_phonall', 'knn']
0.6016949152542372
['eng_all_phonred', 'knn']
0.5738498789346247
['vae', 'knn']
0.7039951573849879
['classall', 'knn']
0.6918886198547216
['classred', 'knn']
0.6204600484261502
['syllall', 'knn']
0.7687651331719128
['syllred', 'knn']
0.7512106537530266
['phonall', 'knn']
0.75544794188862
['phonred', 'knn']
0.74455205811138


In [19]:
# Results Utterance-Wise Also Valid

for c in range(len(clfs)):

    for b in range(len(modes)):

        mode = modes[b]

        print([mode,clfs[c]])

        accuracies = np.load('../../results/' + mode + '/accuracies_also_valid.npy')

        frame_sizes = ['1024']

        accuracies = np.mean(accuracies,axis=-1)
        accuracies[0,:,0] *= utterance_wise_weights
        accuracies_allpart = np.sum(accuracies,axis=-2)
        
        for a in range(len(frame_sizes)):
            print(accuracies_allpart[a,c])

['eng_mfcc_env', 'knn']
0.679176755447942
['eng_all_classall', 'knn']
0.7052058111380145
['eng_all_classred', 'knn']
0.7015738498789346
['eng_all_syllall', 'knn']
0.6700968523002422
['eng_all_syllred', 'knn']
0.6846246973365617
['eng_all_phonall', 'knn']
0.608958837772397
['eng_all_phonred', 'knn']
0.5865617433414043
['vae', 'knn']
0.7106537530266344
['classall', 'knn']
0.7003631961259081
['classred', 'knn']
0.6222760290556901
['syllall', 'knn']
0.7717917675544794
['syllred', 'knn']
0.7651331719128329
['phonall', 'knn']
0.7590799031476998
['phonred', 'knn']
0.7487893462469735
